# 01 — EDA y Preprocesamiento
Documento en tercera persona. Este notebook inspecciona calidad de datos (nulos, duplicados), distribuciones y prepara tablas procesadas.

In [ ]:
import yaml, polars as pl
from src.data_utils import load_mal_tables
from pathlib import Path

with open('config/config.yaml') as f:
    cfg = yaml.safe_load(f)

dfs = load_mal_tables(cfg['paths']['raw_dir'])
for k,v in dfs.items():
    print(k, v.shape)
